___

# CLEANING

In [1]:
import json
import pickle
import os
import re
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
issue_datasets = {}
activities = ["exports", "imports", "production"]
commodities = []
issue_sheets = ["exports_cobalt_1980_1989.xlsx", "exports_cobalt_1990_1999.xlsx", "exports_cobalt_2000_20009.xlsx"]
test_sheet = ['./aggregated_exports/exports_cobalt.csv', './aggregated_exports/exports_copper.csv', './aggregated_exports/exports_diamond.csv', './aggregated_exports/exports_gold.csv', './aggregated_exports/exports_iron_steel_and_ferro-alloys.csv', './aggregated_exports/exports_lithium.csv', './aggregated_exports/exports_magnesite.csv', './aggregated_exports/exports_magnesite_and_magnesia.csv', './aggregated_exports/exports_nickel.csv', './aggregated_exports/exports_sillimanite_minerals.csv', './aggregated_exports/exports_silver.csv', './aggregated_exports/exports_sulphur_and_pyrites.csv', './aggregated_imports/imports_copper.csv', './aggregated_imports/imports_diamond.csv', './aggregated_imports/imports_gold.csv', './aggregated_imports/imports_iron_steel_and_ferro-alloys.csv', './aggregated_imports/imports_nickel.csv', './aggregated_imports/imports_sillimanite_minerals.csv', './aggregated_imports/imports_sulphur_and_pyrites.csv', './aggregated_imports/imports_tantalum_and_niobium.csv', './aggregated_production/production_cement.csv', './aggregated_production/production_cement__clinker.csv', './aggregated_production/production_cement_finished.csv', './aggregated_production/production_ferro-alloys.csv']
strings = []

file_name = "countries.pkl"

open_file = open(file_name, "rb")
countries = pickle.load(open_file)
open_file.close()

countries = sorted(countries)

with open('issue_datasets.json', 'r') as fp:
    issue_dict = json.load(fp)

In [3]:
def get_issue_sheets(issue_dict):
    issue_datasets = list(issue_dict.keys())
    test_sheet = [path.split("/")[-1] for path in test_sheet]
    issue_sheets_by_year

def check_issue_character(df):
    ignore_strings = []
    issue_strings = [c for c in strings]
    issue_characters = [")", "]", "^", "*"] 
    for issue_character in issue_characters:
        for issue_string in issue_strings:
            if issue_character in issue_string:
                if issue_string not in ignore_strings:
                    ignore_strings.append(issue_string)
    append_strings = [c for c in issue_strings if c not in ignore_strings]
    print(strings)
    print(f"Ignoring: {ignore_strings}")
    print(f"Appending: {append_strings}")        

In [4]:
def clean_issue_sheets(df):
    df = pd.read_excel(path, header=1, engine="openpyxl")
    issue_value = df[df["\n\tCountry"] == "Cuba"][year_value].values[0]
    subcommodity_value = df[(df["\n\tCountry"] == "Cuba") & (df[year_value] == issue_value)]["Sub-commodity"].values[0]
    row_label = df[(df["\n\tCountry"] == "Cuba") & (df[year_value] == issue_value)].index.values[0]
    new_value = f"{subcommodity_value.strip()}, {issue_value.strip()}"
    df.at[row_label, "Sub-commodity"] = new_value
    df.iloc[row_label, 2:] = df.iloc[row_label, 2:].shift(-1, axis=0)
    return df

def clean_df(path, activity, commodity):
    df = pd.read_excel(path, header=1, engine="openpyxl")
    # Clean column names
    df = df.rename(columns = {'\n\tCountry':'Country'})
    # Add activity and commodity column
    df["Activity"] = activity
    df["Commodity"] = commodity
    # Try loading DataFrame
    df= try_loading_df(df, activity, commodity)
    #print(df)
    string_columns = ["Country", "Activity", "Commodity"]
    #amount_columns = [col for col in df.columns if col not in string_columns]
    df[string_columns] = df[string_columns].apply(lambda s: s.str.strip())
    
    return df

In [5]:
def try_loading_df(df, activity, commodity):
    if type(df) == str:
        df = pd.read_csv(df, header=0)

    #print(df.head())
    df = df[df['Country'].isin(countries)]
    flag = 0
    while flag != 1:
        try:
            first_columns = ["Country", "Activity", "Commodity", "Sub-commodity"]
            try_df = df.copy()
            try_df = try_df[first_columns + [col for col in try_df.columns if col not in first_columns]]
            # Shift Unnamed values to the left
            print(f"Trying to shift axis for {path}")
            try_df.iloc[:, 4:] = try_df.iloc[:, 4:].T.shift(-1,axis=0).T
            # Remove Unnamed columns
            try_df = try_df[try_df.columns.drop(list(try_df.filter(regex='Unnamed')))]
            # Clean year column titles
            try_df.columns = try_df.columns.astype('str').str.replace("\.0|\\n\\t", "", regex=True)
            # Try to melt dataframe
            print(f"Trying to melt {path}")
            melted_df = try_df.melt(id_vars=["Country", "Activity", "Commodity", "Sub-commodity"], var_name="Year", value_name="Amount")
            melted_df["Amount"] = melted_df["Amount"].astype(float)
            print(f"Loaded DataFrame successfully")
            flag = 1
            return try_df
        except Exception as e:
            print(f"WARNING! {path} could not be loaded: {repr(e)}")
            #print(df.head())
            issue_string = str(e).split("'")[1]
            if issue_string not in strings:
                strings.append(issue_string)
            issue_datasets[path] = e
            ignore_strings = []
            issue_characters = [")", "]", "^", "*", "#", np.nan]
            unique_values = df.iloc[:, 2].unique()
            float_values = []
            #print(df.iloc[:, 2])
            for value in unique_values:
                try:
                    float(value)
                    float_values.append(value)
                    continue
                except:
                    for issue_character in issue_characters:
                        if (pd.notnull(issue_character)) and (issue_character in value):
                            ignore_strings.append(value)
                            print(f"{issue_character} detected in {value}")
                            #df[df.iloc[:, 2] == value].iloc[:2] = np.nan
            print(ignore_strings)
            append_strings = [value for value in unique_values if (value not in ignore_strings) and (value not in float_values) and (pd.notnull(value))]
            print(append_strings)
            # If value is not an issue string, append it to previous column
            for value in append_strings:
                mask = df.iloc[:, 2] == value
                print(f"Appending <{value}> to <{commodity}>...")
                df.loc[mask, "Sub-commodity"] = df[mask]["Sub-commodity"].apply(lambda s: f"{s.strip()}, {value.strip()}" if pd.notnull(s) else None)
                #df['Sub-commodity'] = df['Sub-commodity'].apply(lambda s: f"{s}, {value.strip()}")
                print(f"Shifting rows containing {value.strip()}")
                cols_to_shift = df.columns[2:-2]
                # print(df.loc[mask, cols_to_shift])
                df.loc[mask, cols_to_shift] = df.loc[mask, cols_to_shift].T.shift(-1,axis=0).T
                #df.iloc[:, 2:] = df.iloc[:, 2:].T.shift(-1,axis=0)
                df[["Sub-commodity"]] = df[["Sub-commodity"]].apply(lambda s: s.str.strip())
            continue
                
            

In [6]:
for activity in activities:
    for sheet in os.listdir(f"./{activity}/"):
        path = f"./{activity}/{sheet}"
        files_to_ignore = [".DS_Store", ".ipynb_checkpoints"]
        if any(file in path for file in files_to_ignore):
            continue
        save_name = re.sub("\.|\/|imports|exports|production|xlsx", "", path)[1:].replace(", ", "_").replace(" ", "_")
        print(save_name)
        commodity = re.sub("\_\d.*$", "", save_name)
        commodities.append(commodity)
        # outer_flag = 0
        # while outer_flag != 1:
        #     try:
        #         df = clean_df(path, commodity)
        #         outer_flag = 1
        #     except:
        #         print(e)
        df = clean_df(path, activity, commodity)
        #print("Printing DataFrame before writing")
        #print(df.head())
        save_path = f"/app/mineral_resources/cleaned_{activity}/{save_name}.csv"
        print(f"Saving DataFrame to {save_path}")
        print("\n\n")
        df.to_csv(save_path, index=False)

aggregates_primary_1990_1999
Trying to shift axis for ./exports/exports_aggregates, primary_1990_1999.xlsx
Trying to melt ./exports/exports_aggregates, primary_1990_1999.xlsx
Loaded DataFrame successfully
Saving DataFrame to /analytics_env/resources/mineral_resources/cleaned_exports/aggregates_primary_1990_1999.csv



aggregates_primary_2000_2009
Trying to shift axis for ./exports/exports_aggregates, primary_2000_2009.xlsx
Trying to melt ./exports/exports_aggregates, primary_2000_2009.xlsx
Loaded DataFrame successfully
Saving DataFrame to /analytics_env/resources/mineral_resources/cleaned_exports/aggregates_primary_2000_2009.csv



aggregates_primary_2010_2019
Trying to shift axis for ./exports/exports_aggregates, primary_2010_2019.xlsx
Trying to melt ./exports/exports_aggregates, primary_2010_2019.xlsx
Loaded DataFrame successfully
Saving DataFrame to /analytics_env/resources/mineral_resources/cleaned_exports/aggregates_primary_2010_2019.csv



aggregates_primary_2020_2020
Trying to s

In [7]:
strings

['(b)*',
 '(b)',
 ' cement & refined',
 '(l)',
 '(e)*',
 '(w)',
 '(a)',
 '^',
 '(e)',
 '(c)*',
 ' billets                           ',
 '(f)*',
 '(f)(l)*',
 '(c)',
 ' mattes etc',
 '(i)',
 ' dinas earth           ',
 '(f)',
 '*',
 '(g)',
 '(a)*',
 '(d)',
 '(h)',
 ' sinters etc',
 ' dinas earth',
 '#',
 ' Fe-Mo',
 ' Fe-V                          ',
 '(a)(h)']

In [8]:
ignore_strings = []

In [9]:
issue_strings = [c for c in strings]

In [10]:
issue_strings

['(b)*',
 '(b)',
 ' cement & refined',
 '(l)',
 '(e)*',
 '(w)',
 '(a)',
 '^',
 '(e)',
 '(c)*',
 ' billets                           ',
 '(f)*',
 '(f)(l)*',
 '(c)',
 ' mattes etc',
 '(i)',
 ' dinas earth           ',
 '(f)',
 '*',
 '(g)',
 '(a)*',
 '(d)',
 '(h)',
 ' sinters etc',
 ' dinas earth',
 '#',
 ' Fe-Mo',
 ' Fe-V                          ',
 '(a)(h)']

In [11]:
issue_characters = [")", "]", "^", "*"] 
for issue_character in issue_characters:
    for issue_string in issue_strings:
        if issue_character in issue_string:
            if issue_string not in ignore_strings:
                print(issue_string)
                ignore_strings.append(issue_string)


(b)*
(b)
(l)
(e)*
(w)
(a)
(e)
(c)*
(f)*
(f)(l)*
(c)
(i)
(f)
(g)
(a)*
(d)
(h)
(a)(h)
^
*


In [12]:
ignore_strings

['(b)*',
 '(b)',
 '(l)',
 '(e)*',
 '(w)',
 '(a)',
 '(e)',
 '(c)*',
 '(f)*',
 '(f)(l)*',
 '(c)',
 '(i)',
 '(f)',
 '(g)',
 '(a)*',
 '(d)',
 '(h)',
 '(a)(h)',
 '^',
 '*']

In [13]:
append_strings = [c for c in issue_strings if c not in ignore_strings]

In [14]:
append_strings

[' cement & refined',
 ' billets                           ',
 ' mattes etc',
 ' dinas earth           ',
 ' sinters etc',
 ' dinas earth',
 '#',
 ' Fe-Mo',
 ' Fe-V                          ']

In [15]:
ignore_strings

['(b)*',
 '(b)',
 '(l)',
 '(e)*',
 '(w)',
 '(a)',
 '(e)',
 '(c)*',
 '(f)*',
 '(f)(l)*',
 '(c)',
 '(i)',
 '(f)',
 '(g)',
 '(a)*',
 '(d)',
 '(h)',
 '(a)(h)',
 '^',
 '*']

In [16]:
any(issue_characters)
issue_characters

[')', ']', '^', '*']

In [17]:
commodities = set(commodities)
commodities

{'aggregates_primary',
 'alumina',
 'aluminium_primary',
 'antimony',
 'antimony_mine',
 'arsenic',
 'arsenic_white',
 'asbestos',
 'asbestos_unmanufactured',
 'barytes',
 'bauxite',
 'bauxite_alumina_and_aluminium',
 "bentonite_and_fuller's_earth",
 'beryl',
 'bismuth',
 'bismuth_mine',
 'borates',
 'bromine',
 'cadmium',
 'cement',
 'cement__clinker',
 'cement_finished',
 'chromium',
 'chromium_ores_and_concentrates',
 'coal',
 'cobalt',
 'cobalt_mine',
 'cobalt_refined',
 'copper',
 'copper_mine',
 'copper_refined',
 'copper_smelter',
 'diamond',
 'diatomite',
 'feldspar',
 'ferro-alloys',
 'fluorspar',
 'gallium_primary',
 'gemstones',
 'germanium_metal',
 'gold',
 'gold_mine',
 'graphite',
 'gypsum_and_plaster',
 'hafnium_refined',
 'helium',
 'indium_refinery',
 'iodine',
 'iron_ore',
 'iron_pig',
 'iron_steel_and_ferro-alloys',
 'kaolin',
 'lead',
 'lead_mine',
 'lead_refined',
 'lithium',
 'lithium_minerals',
 'magnesite',
 'magnesite_and_magnesia',
 'magnesium_metal_primary',


In [18]:
file_name = "minerals.pkl"

open_file = open(file_name, "wb")
pickle.dump(commodities, open_file)
open_file.close()